## Preprocessing

In [55]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import os
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [56]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [57]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [58]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [59]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
cutoff_value = 500
application_types_to_replace = application_type_counts[application_type_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [60]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [61]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()

# Filter the counts to only include those greater than 1
classification_counts_filtered = classification_counts[classification_counts > 1]
print(classification_counts_filtered)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [62]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff_value = 1000
classifications_to_replace = classification_counts[classification_counts < cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [63]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, drop_first=True)
application_df.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,1,108590,1,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,1,5000,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1,6692,1,False,False,True,False,False,False,False,...,True,False,True,False,False,False,False,False,False,False
4,1,142590,1,False,False,True,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False


In [64]:
X = application_df.drop('IS_SUCCESSFUL', axis=1)
y = application_df['IS_SUCCESSFUL']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.25)


In [65]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [66]:
# Importing necessary Keras modules
from tensorflow.keras.layers import Dense

# Number of input features
input_features = len(X_train_scaled[0])

# Initializing the Sequential model
nn = tf.keras.models.Sequential()

# First hidden layer with 80 nodes and ReLU activation
nn.add(Dense(units=80, input_dim=input_features, activation='relu'))

# Second hidden layer with 30 nodes and ReLU activation
nn.add(Dense(units=30, activation='relu'))

# Output layer with 1 node and sigmoid activation
nn.add(Dense(units=1, activation='sigmoid'))

# Displaying the structure of the model
nn.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                2960      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5421 (21.18 KB)
Trainable params: 5421 (21.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [67]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [68]:
# Train the model
# Step 8: Create a callback that saves the model's weights every five epochs.
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/", exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Step 9: Train the model (we're re-training for the sake of this demonstration)
fit_model = nn.fit(
    X_train_scaled, 
    y_train, 
    epochs=100,
    callbacks=[cp_callback]
)


Epoch 1/100
804/804 [==============================] - 2s 960us/step - loss: 0.5732 - accuracy: 0.7205
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5571 - accuracy: 0.7275
Epoch 3/100
804/804 [==============================] - 1s 964us/step - loss: 0.5547 - accuracy: 0.7277
Epoch 4/100
804/804 [==============================] - 1s 827us/step - loss: 0.5516 - accuracy: 0.7296
Epoch 5/100
791/804 [============================>.] - ETA: 0s - loss: 0.5505 - accuracy: 0.7308
Epoch 5: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 1s 961us/step - loss: 0.5503 - accuracy: 0.7313
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5499 - accuracy: 0.7314
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5493 - accuracy: 0.7327
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5480 - accuracy: 0.7330
Epoch 9/100
804/804 [=========================

In [69]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5629 - accuracy: 0.7263 - 231ms/epoch - 860us/step
Loss: 0.5629037022590637, Accuracy: 0.7262973785400391


In [70]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

c:\Users\perry\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Model Version 1.1 
## Increased epoch count, extra neurons/layer and and additional hidden layer

In [71]:
# Number of input features
input_features = len(X_train_scaled[0])

# Initializing the Sequential model
nn_1 = tf.keras.models.Sequential(name="nn_V1.1")

# First hidden layer with 100 nodes and ReLU activation
nn_1.add(Dense(units=100, input_dim=input_features, activation='relu'))

# Second hidden layer with 50 nodes and ReLU activation
nn_1.add(Dense(units=50, activation='relu'))

# Third hidden layer with 50 nodes and ReLU activation
nn_1.add(Dense(units=50, activation='sigmoid'))

# Output layer with 1 node and sigmoid activation
nn_1.add(Dense(units=1, activation='sigmoid'))

# Displaying the structure of the model
nn_1.summary()


Model: "nn_V1.1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               3700      
                                                                 
 dense_7 (Dense)             (None, 50)                5050      
                                                                 
 dense_8 (Dense)             (None, 50)                2550      
                                                                 
 dense_9 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11351 (44.34 KB)
Trainable params: 11351 (44.34 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [72]:
# Compile the model
nn_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [73]:
# Train the model

# Define the checkpoint path and filenames
os.makedirs("checkpointsV1.1/", exist_ok=True)
checkpoint_path = "checkpointsV1.1/weightsV1.1.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Step 9: Train the model (we're re-training for the sake of this demonstration)
fit_model = nn_1.fit(
    X_train_scaled, 
    y_train, 
    epochs=150,
    callbacks=[cp_callback]
)


Epoch 1/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5703 - accuracy: 0.7160
Epoch 2/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5557 - accuracy: 0.7270
Epoch 3/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5523 - accuracy: 0.7298
Epoch 4/150
804/804 [==============================] - 1s 1ms/step - loss: 0.5507 - accuracy: 0.7303
Epoch 5/150
792/804 [============================>.] - ETA: 0s - loss: 0.5502 - accuracy: 0.7317
Epoch 5: saving model to checkpointsV1.1\weightsV1.1.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5500 - accuracy: 0.7317
Epoch 6/150
804/804 [==============================] - 1s 842us/step - loss: 0.5492 - accuracy: 0.7325
Epoch 7/150
804/804 [==============================] - 1s 838us/step - loss: 0.5485 - accuracy: 0.7319
Epoch 8/150
804/804 [==============================] - 1s 907us/step - loss: 0.5474 - accuracy: 0.7317
Epoch 9/150
804/804 [===================

In [74]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5877 - accuracy: 0.7285 - 466ms/epoch - 2ms/step
Loss: 0.5876731276512146, Accuracy: 0.7285131216049194


In [75]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity1.1.h5")

# Model Version 1.2
##

In [76]:
from tensorflow.keras.layers import LeakyReLU

# Number of input features
input_features = len(X_train_scaled[0])

# Initializing the Sequential model
nn_2 = tf.keras.models.Sequential(name="nn_V1.2")

# First hidden layer with 100 nodes and ReLU activation
nn_2.add(Dense(units=150, input_dim=input_features, activation='relu'))

# Second hidden layer with 50 nodes and ReLU activation
nn_2.add(Dense(units=100, activation='relu'))

# Third hidden layer with 50 nodes and ReLU activation
nn_2.add(Dense(units=128))
nn_2.add(LeakyReLU(alpha=0.01))  # alpha is the negative slope coefficient

# Output layer with 1 node and sigmoid activation
nn_2.add(Dense(units=1, activation='sigmoid'))

# Displaying the structure of the model
nn_2.summary()

# Compile the model
nn_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model

# Define the checkpoint path and filenames
os.makedirs("checkpointsV1.2/", exist_ok=True)
checkpoint_path = "checkpointsV1.2/weightsV1.2.{epoch:02d}.hdf5"


# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Step 9: Train the model
fit_model = nn_2.fit(
    X_train_scaled, 
    y_train, 
    epochs=150,
    callbacks=[cp_callback]
)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn.save("AlphabetSoupCharity1.2.h5")

Model: "nn_V1.2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 150)               5550      
                                                                 
 dense_11 (Dense)            (None, 100)               15100     
                                                                 
 dense_12 (Dense)            (None, 128)               12928     
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128)               0         
                                                                 
 dense_13 (Dense)            (None, 1)                 129       
                                                                 
Total params: 33707 (131.67 KB)
Trainable params: 33707 (131.67 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/150
804/804 [==============================] - 3s 2ms/step - loss: 0.5696 - accuracy: 0.7205
Epoch 2/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5567 - accuracy: 0.7274
Epoch 3/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5536 - accuracy: 0.7298
Epoch 4/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5515 - accuracy: 0.7310
Epoch 5/150
786/804 [============================>.] - ETA: 0s - loss: 0.5500 - accuracy: 0.7316
Epoch 5: saving model to checkpointsV1.2\weightsV1.2.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5506 - accuracy: 0.7309
Epoch 6/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5498 - accuracy: 0.7322
Epoch 7/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5484 - accuracy: 0.7314
Epoch 8/150
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7334
Epoch 9/150
804/804 [=========================

# Model Version 1.3

In [77]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_3 = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_3.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=input_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_3.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_3.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_3

In [78]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [79]:
# Run the kerastuner search for best hyperparameters without callbacks
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))


Trial 60 Complete [00h 00m 17s]
val_accuracy: 0.5343440175056458

Best val_accuracy So Far: 0.7338775396347046
Total elapsed time: 00h 07m 21s
INFO:tensorflow:Oracle triggered exit


In [80]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 7,
 'num_layers': 4,
 'units_0': 5,
 'units_1': 3,
 'units_2': 1,
 'units_3': 3,
 'units_4': 1,
 'units_5': 9,
 'tuner/epochs': 20,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 0,
 'tuner/round': 0}

In [81]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5545 - accuracy: 0.7339 - 273ms/epoch - 1ms/step
Loss: 0.5544507503509521, Accuracy: 0.7338775396347046


In [82]:
def build_best_model(hp_values):
    nn_best = tf.keras.models.Sequential()

    # Use the best activation function from hyperparameters
    activation = hp_values['activation']
    
    # Use the best number of neurons for the first layer
    nn_best.add(tf.keras.layers.Dense(units=hp_values['first_units'], activation=activation, input_dim=input_features))

    # Use the best number of hidden layers and neurons for each hidden layer
    for i in range(hp_values['num_layers']):
        nn_best.add(tf.keras.layers.Dense(units=hp_values['units_' + str(i)], activation=activation))
    
    nn_best.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_best.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_best

# Use the best hyperparameters to construct the model
best_model = build_best_model(best_hyper.values)

# Train the model

# Define the checkpoint path and filenames
os.makedirs("checkpointsV1.3/", exist_ok=True)
checkpoint_path = "checkpointsV1.3/weightsV1.3.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5
)

# Step 9: Train the model
fit_model = best_model.fit(
    X_train_scaled, 
    y_train, 
    epochs=150,
    callbacks=[cp_callback]
)

# Evaluate the model using the test data
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn.save("AlphabetSoupCharity1.3.h5")

Epoch 1/150
804/804 [==============================] - 1s 853us/step - loss: 0.6034 - accuracy: 0.6880
Epoch 2/150
804/804 [==============================] - 1s 838us/step - loss: 0.5632 - accuracy: 0.7271
Epoch 3/150
804/804 [==============================] - 1s 889us/step - loss: 0.5593 - accuracy: 0.7288
Epoch 4/150
804/804 [==============================] - 1s 808us/step - loss: 0.5579 - accuracy: 0.7288
Epoch 5/150
757/804 [===========================>..] - ETA: 0s - loss: 0.5556 - accuracy: 0.7310
Epoch 5: saving model to checkpointsV1.3\weightsV1.3.05.hdf5
804/804 [==============================] - 1s 901us/step - loss: 0.5567 - accuracy: 0.7301
Epoch 6/150
804/804 [==============================] - 1s 875us/step - loss: 0.5561 - accuracy: 0.7299
Epoch 7/150
804/804 [==============================] - 1s 891us/step - loss: 0.5554 - accuracy: 0.7298
Epoch 8/150
804/804 [==============================] - 1s 864us/step - loss: 0.5552 - accuracy: 0.7296
Epoch 9/150
804/804 [=========